# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,-3.22,83,100,2.57,US,1705411224
1,1,polyarnyy,69.1989,33.4478,-14.46,94,100,4.43,RU,1705411162
2,2,kavieng,-2.5744,150.7967,28.18,79,100,3.74,PG,1705411227
3,3,vilyuchinsk,52.9306,158.4028,-7.85,92,93,4.02,RU,1705411228
4,4,port elizabeth,-33.9180,25.5701,25.61,67,20,6.17,ZA,1705411187


In [3]:
# Display specific columns that can indicate units(mr results differ from example, mints might not match, remember to check this)
print(city_data_df[['Max Temp', 'Wind Speed']].head(20))



    Max Temp  Wind Speed
0      -3.22        2.57
1     -14.46        4.43
2      28.18        3.74
3      -7.85        4.02
4      25.61        6.17
5      12.12        5.34
6      25.71        2.23
7      23.53        7.45
8      28.98        6.84
9      26.12        8.45
10     24.88        2.54
11     -5.27        1.10
12     -3.42        7.86
13      9.25        3.09
14      5.76        7.82
15    -30.88        3.09
16     26.57        4.30
17     16.53       14.31
18      1.17        1.79
19     24.75        1.79


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot using hvplot
map_plot_1 = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,  # Enable geographic projections
    tiles='OSM',  # Use OpenStreetMap tiles
    frame_width=800,
    frame_height=600,
    size='Humidity',  # Size of points based on 'Humidity'
    scale=1,  # Scale factor for the size
    color='City',  # Color of points based on 'City'
    hover_cols=['City', 'Humidity']  # Columns to show on hover
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# # Narrow down cities that fit criteria and drop any results with null values
# # Apply filters for ideal weather conditions
# ideal_weather_df = city_data_df[
#     (city_data_df['Max Temp'] > 21) & 
#     (city_data_df['Max Temp'] < 27) &
#     (city_data_df['Wind Speed'] < 4.5) &
#     (city_data_df['Cloudiness'] == 0)
# ]

# # Drop any rows with null values
# ideal_weather_df = ideal_weather_df.dropna()

# # Display sample data
# ideal_weather_df.head()

# Convert temperature from Celsius to Fahrenheit (if your data is in Celsius)
city_data_df['Max Temp F'] = (city_data_df['Max Temp'] * 9/5) + 32

# Convert wind speed from meters per second to miles per hour (if your data is in meters per second)
city_data_df['Wind Speed mph'] = city_data_df['Wind Speed'] * 2.237

# Assuming ideal conditions in Fahrenheit and miles per hour
ideal_temp_min_f = 70  # equivalent to 21 degrees Celsius
ideal_temp_max_f = 80.6  # equivalent to 27 degrees Celsius
ideal_wind_speed_max_mph = 10  # equivalent to 4.5 m/s

# Apply filters for ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp F'] > ideal_temp_min_f) & 
    (city_data_df['Max Temp F'] < ideal_temp_max_f) &
    (city_data_df['Wind Speed mph'] < ideal_wind_speed_max_mph) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head(25)





,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp F,Wind Speed mph
46,46,hawaiian paradise park,19.5933,-154.9731,22.80,97,0,3.09,US,1705411314,73.040,6.91233
220,220,tamanrasset,22.7850,5.5228,24.95,4,0,1.54,DZ,1705411604,76.910,3.44498
231,231,sao gabriel da cachoeira,-0.1303,-67.0892,26.89,89,0,4.12,BR,1705411620,80.402,9.21644
268,268,illapel,-31.6308,-71.1653,22.23,42,0,1.15,CL,1705411675,72.014,2.57255
295,295,pisco,-13.7000,-76.2167,23.03,83,0,2.57,PE,1705411720,73.454,5.74909
395,395,negombo,7.2083,79.8358,26.03,78,0,1.54,LK,1705411894,78.854,3.44498
477,477,savanna-la-mar,18.2190,-78.1332,25.97,63,0,1.93,JM,1705412030,78.746,4.31741
532,532,callao,-12.0667,-77.1500,23.16,83,0,4.12,PE,1705411842,73.688,9.21644
551,551,hilo,19.7297,-155.0900,22.92,77,0,3.09,US,1705412148,73.256,6.91233
570,570,santa clara,22.4000,-79.9667,21.81,94,0,2.06,CU,1705412175,71.258,4.60822


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
46,hawaiian paradise park,US,19.5933,-154.9731,97,
220,tamanrasset,DZ,22.7850,5.5228,4,
231,sao gabriel da cachoeira,BR,-0.1303,-67.0892,89,
268,illapel,CL,-31.6308,-71.1653,42,
295,pisco,PE,-13.7000,-76.2167,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 50000
geoapify_key = 'YOUR_GEOAPIFY_API_KEY'  # Replace with your Geoapify API key

params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel',
    'limit': 1  # Limit to the first result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    print(response.url)  # Print the request URL for debugging
    print(response.status_code)  # Print the status code of the response

    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
https://api.geoapify.com/v2/places?apiKey=YOUR_GEOAPIFY_API_KEY&categories=accommodation.hotel&limit=1&filter=circle%3A-154.9731%2C19.5933%2C50000&bias=proximity%3A-154.9731%2C19.5933
401
hawaiian paradise park - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?apiKey=YOUR_GEOAPIFY_API_KEY&categories=accommodation.hotel&limit=1&filter=circle%3A5.5228%2C22.785%2C50000&bias=proximity%3A5.5228%2C22.785
401
tamanrasset - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?apiKey=YOUR_GEOAPIFY_API_KEY&categories=accommodation.hotel&limit=1&filter=circle%3A-67.0892%2C-0.1303%2C50000&bias=proximity%3A-67.0892%2C-0.1303
401
sao gabriel da cachoeira - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?apiKey=YOUR_GEOAPIFY_API_KEY&categories=accommodation.hotel&limit=1&filter=circle%3A-71.1653%2C-31.6308%2C50000&bias=proximity%3A-71.1653%2C-31.6308
401
illapel - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?apiKe

,City,Country,Lat,Lng,Humidity,Hotel Name
46,hawaiian paradise park,US,19.5933,-154.9731,97,No hotel found
220,tamanrasset,DZ,22.7850,5.5228,4,No hotel found
231,sao gabriel da cachoeira,BR,-0.1303,-67.0892,89,No hotel found
268,illapel,CL,-31.6308,-71.1653,42,No hotel found
295,pisco,PE,-13.7000,-76.2167,83,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size=10,
    hover_cols=['City', 'Hotel Name', 'Country'],  # Columns to show on hover
    color='blue'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country)